In [62]:
import numpy as np
import pandas as pd

vocab_dir = '../vocabulary_various/'
data_dir='../data/'
rxnorm_relations_table_dir='../RxNorm_relationships_tables/'

import os
directory = rxnorm_relations_table_dir
if not os.path.exists(directory):
    os.makedirs(directory)

In [23]:
c=(pd.read_csv(vocab_dir+'CONCEPT.csv',sep='\t').
  query('vocabulary_id=="RxNorm"').
  loc[:,['concept_id','concept_code','concept_name','concept_class_id']].
  drop_duplicates()
  )
c['concept_id'] = c['concept_id'].astype(int)
cr=(pd.read_csv(vocab_dir+'CONCEPT_RELATIONSHIP.csv',sep='\t').
    loc[:,['concept_id_1','concept_id_2','relationship_id']].
    drop_duplicates()
   )

In [34]:
cr_verbose = \
(cr.
 set_index('concept_id_1').
 join(
     c.
     set_index('concept_id').
     rename(columns={'concept_name' : 'concept_name_1',
                     'concept_code' : 'concept_code_1',
                     'concept_class_id' : 'concept_class_id_1'})
 ).
 dropna().
 rename_axis('concept_id_1').
 reset_index().
 set_index('concept_id_2').
 join(
     c.
     set_index('concept_id').
     rename(columns={'concept_name' : 'concept_name_2',
                     'concept_code' : 'concept_code_2',
                     'concept_class_id' : 'concept_class_id_2'})
 ).
 dropna().
 rename_axis('concept_id_2').
 reset_index().
 rename(columns={'relationship_id' : 'relationship_id_12'})
)
cr_verbose = cr_verbose.reindex(sorted(cr_verbose.columns),axis=1)
print(cr_verbose.shape)
cr_verbose.head()

(1975234, 9)


,concept_class_id_1,concept_class_id_2,concept_code_1,concept_code_2,concept_id_1,concept_id_2,concept_name_1,concept_name_2,relationship_id_12
0,Ingredient,Ingredient,26744,26744,501343,501343,hepatitis B immune globulin,hepatitis B immune globulin,Maps to
1,Ingredient,Ingredient,26744,26744,501343,501343,hepatitis B immune globulin,hepatitis B immune globulin,Mapped from
2,Clinical Drug Comp,Ingredient,353465,26744,501345,501343,hepatitis B immune globulin 217 UNT/ML,hepatitis B immune globulin,RxNorm has ing
3,Clinical Drug Comp,Ingredient,825152,26744,501421,501343,hepatitis B immune globulin 220 UNT/ML,hepatitis B immune globulin,RxNorm has ing
4,Brand Name,Ingredient,215578,26744,19056007,501343,Bayhep B,hepatitis B immune globulin,Brand name of


In [35]:
cr_verbose.relationship_id_12.value_counts()

RxNorm is a             211619
RxNorm inverse is a     211619
Maps to                 191811
Mapped from             191811
Has brand name          111256
Brand name of           111256
RxNorm dose form of     102392
RxNorm has dose form    102392
Tradename of            101171
Has tradename           101171
RxNorm has ing           84908
RxNorm ing of            84908
Consists of              81010
Constitutes              81010
Concept replaces         40782
Concept replaced by      40782
Has dose form group      38320
Dose form group of       38320
Has precise ing          12984
Precise ing of           12984
Has quantified form       5977
Quantified form of        5977
Has form                  2891
Form of                   2891
Contained in              2495
Contains                  2495
Component of                 1
Has component                1
Name: relationship_id_12, dtype: int64

https://www.nlm.nih.gov/research/umls/rxnorm/overview.html

![](../docs/imgs/RxNorm_Drug_Relationships.png)

Each SAB=RXNORM normalized name follows a simple pattern using the ingredient, strength, and dose form. For branded drugs, SAB=RXNORM includes the brand name, and for packs of drugs, SAB=RXNORM includes the quantities of each drug.

- Generic drugs - Ingredient Strength Dose Form

- Branded drugs - Ingredient Strength Dose Form [Brand Name]

- Generic drug packs - {# (Ingredient Strength Dose Form) / # (Ingredient Strength Dose Form)} Pack
- Branded drug packs - {# (Ingredient Strength Dose Form) / # (Ingredient Strength Dose Form)} Pack [Brand Name]

Ingredient: Ingredients generally use the United States Adopted Name (USAN). The Basis of Strength Substance (BoSS) is used in the fully normalized name (e.g., warfarin sodium vs warfarin in 'Warfarin Sodium 1 MG Oral Tablet').

Strength: SAB=RXNORM bases the strength on the active ingredient(s), which may be either the precise (salt or ester) or the base ingredient of the drug. Strengths use the "milli" metric system equivalents (e.g., milligrams, milliliters). Liquids are expressed per milliliter. When a powder can be reconstituted to a range of liquid concentrations, the highest concentration in the range is used for the normalized form.

Dose Form: SAB=RXNORM uses a set of dose forms based upon those included in the Health Level 7 (HL7) data standard.

Brand name: SAB=RXNORM usually represents brand names as seen on the packaging (e.g., Zoloft, Tylenol Cold Relief, and Advair 115/21). In cases where the name on the packaging includes extra information that represents information beyond ingredient or strength, SAB=RXNORM removes that additional information (e.g., Lamisil instead of Lamisil AT and Spiriva instead of Spiriva HandiHaler).

Clinical Drug Comp == IN + Strength

Clinical Drug Form == IN + Dose Form

Clinical Drug Group == IN + Dose Form Group

Clinical Drug == IN + Strength + Dose Form

Branded Drug Comp == IN + Strength + Brand Name

Branded Drug Form == IN + Dose Form + Brand Name

Branded Drug == IN + Strength + Dose Form + Brand Name

Has relationships

In [73]:
rids=cr_verbose[cr_verbose.relationship_id_12.str.contains('[Hh]as')].relationship_id_12.unique()
print(rids)
for rid in rids:
    print(rid)
    tmp = cr_verbose.query('relationship_id_12==@rid')
    print(tmp.shape)
    display(tmp.head())
    display(tmp.concept_class_id_1.value_counts())
    display(tmp.concept_class_id_2.value_counts())
    tmp.to_csv(rxnorm_relations_table_dir+'_'.join(rid.split(' '))+'.csv')

['RxNorm has ing' 'Has tradename' 'Has quantified form' 'Has form'
 'Has precise ing' 'Has brand name' 'RxNorm has dose form'
 'Has dose form group' 'Has component']
RxNorm has ing
(84908, 9)


,concept_class_id_1,concept_class_id_2,concept_code_1,concept_code_2,concept_id_1,concept_id_2,concept_name_1,concept_name_2,relationship_id_12
2,Clinical Drug Comp,Ingredient,353465,26744,501345,501343,hepatitis B immune globulin 217 UNT/ML,hepatitis B immune globulin,RxNorm has ing
3,Clinical Drug Comp,Ingredient,825152,26744,501421,501343,hepatitis B immune globulin 220 UNT/ML,hepatitis B immune globulin,RxNorm has ing
8,Clinical Drug Comp,Ingredient,440313,26744,19110172,501343,hepatitis B immune globulin 200 UNT/ML,hepatitis B immune globulin,RxNorm has ing
9,Clinical Drug Comp,Ingredient,440315,26744,19110173,501343,hepatitis B immune globulin 50 UNT/ML,hepatitis B immune globulin,RxNorm has ing
10,Clinical Drug Comp,Ingredient,705903,26744,19126484,501343,hepatitis B immune globulin 312 UNT/ML,hepatitis B immune globulin,RxNorm has ing


Clinical Drug Comp     29575
Clinical Dose Group    29401
Clinical Drug Form     25932
Name: concept_class_id_1, dtype: int64

Ingredient            84883
Precise Ingredient       25
Name: concept_class_id_2, dtype: int64

Has tradename
(101171, 9)


...

Dose Form Group    38320
Name: concept_class_id_2, dtype: int64

Has component
(1, 9)


,concept_class_id_1,concept_class_id_2,concept_code_1,concept_code_2,concept_id_1,concept_id_2,concept_name_1,concept_name_2,relationship_id_12
1364577,Clinical Drug,Clinical Drug Form,199442,373831,19021828,40079589,butylscopolamine bromide 10 MG Oral Tablet,Scopolamine Oral Tablet,Has component


Clinical Drug    1
Name: concept_class_id_1, dtype: int64

Clinical Drug Form    1
Name: concept_class_id_2, dtype: int64

Of relationships

In [74]:
rids=cr_verbose[cr_verbose.relationship_id_12.str.contains('[Oo]f')].relationship_id_12.unique()
print(rids)
for rid in rids:
    print(rid)
    tmp = cr_verbose.query('relationship_id_12==@rid')
    print(tmp.shape)
    display(tmp.head())
    display(tmp.concept_class_id_1.value_counts())
    display(tmp.concept_class_id_2.value_counts())
    tmp.to_csv(rxnorm_relations_table_dir+'_'.join(rid.split(' '))+'.csv')

['Brand name of' 'RxNorm ing of' 'Tradename of' 'Consists of'
 'RxNorm dose form of' 'Precise ing of' 'Form of' 'Quantified form of'
 'Dose form group of' 'Component of']
Brand name of
(111256, 9)


,concept_class_id_1,concept_class_id_2,concept_code_1,concept_code_2,concept_id_1,concept_id_2,concept_name_1,concept_name_2,relationship_id_12
4,Brand Name,Ingredient,215578,26744,19056007,501343,Bayhep B,hepatitis B immune globulin,Brand name of
5,Brand Name,Ingredient,615872,26744,19086257,501343,Hyperhep B,hepatitis B immune globulin,Brand name of
6,Brand Name,Ingredient,643799,26744,19091293,501343,HepaGam B,hepatitis B immune globulin,Brand name of
7,Brand Name,Ingredient,261588,26744,19099582,501343,Nabi-HB,hepatitis B immune globulin,Brand name of
38,Brand Name,Quant Branded Drug,215578,825138,19056007,501415,Bayhep B,0.5 ML hepatitis B immune globulin 217 UNT/ML ...,Brand name of


Brand Name    111256
Name: concept_class_id_1, dtype: int64

Branded Drug          23439
Ingredient            22793
Branded Drug Comp     22511
Branded Dose Group    21600
Branded Drug Form     17854
Quant Branded Drug     2628
Branded Pack            431
Name: concept_class_id_2, dtype: int64

RxNorm ing of
(84908, 9)


...

Branded Dose Group     21600
Clinical Dose Group    16720
Name: concept_class_id_2, dtype: int64

Component of
(1, 9)


,concept_class_id_1,concept_class_id_2,concept_code_1,concept_code_2,concept_id_1,concept_id_2,concept_name_1,concept_name_2,relationship_id_12
341512,Clinical Drug Form,Clinical Drug,373831,199442,40079589,19021828,Scopolamine Oral Tablet,butylscopolamine bromide 10 MG Oral Tablet,Component of


Clinical Drug Form    1
Name: concept_class_id_1, dtype: int64

Clinical Drug    1
Name: concept_class_id_2, dtype: int64

which tables do I need to convert to all classes to ingredient?

In [92]:
term='Ingredient'
grep='[Hh]as'
rids=cr_verbose[cr_verbose.relationship_id_12.str.contains(grep)].relationship_id_12.unique()
for rid in rids:
    tmp = cr_verbose.query('relationship_id_12==@rid')
    if np.any(tmp.concept_class_id_2.str.contains(term)):
        print(rid)
        display(tmp.head())
        display(tmp.concept_class_id_1.value_counts())
        display(tmp.concept_class_id_2.value_counts())

RxNorm has ing


,concept_class_id_1,concept_class_id_2,concept_code_1,concept_code_2,concept_id_1,concept_id_2,concept_name_1,concept_name_2,relationship_id_12
2,Clinical Drug Comp,Ingredient,353465,26744,501345,501343,hepatitis B immune globulin 217 UNT/ML,hepatitis B immune globulin,RxNorm has ing
3,Clinical Drug Comp,Ingredient,825152,26744,501421,501343,hepatitis B immune globulin 220 UNT/ML,hepatitis B immune globulin,RxNorm has ing
8,Clinical Drug Comp,Ingredient,440313,26744,19110172,501343,hepatitis B immune globulin 200 UNT/ML,hepatitis B immune globulin,RxNorm has ing
9,Clinical Drug Comp,Ingredient,440315,26744,19110173,501343,hepatitis B immune globulin 50 UNT/ML,hepatitis B immune globulin,RxNorm has ing
10,Clinical Drug Comp,Ingredient,705903,26744,19126484,501343,hepatitis B immune globulin 312 UNT/ML,hepatitis B immune globulin,RxNorm has ing


Clinical Drug Comp     29575
Clinical Dose Group    29401
Clinical Drug Form     25932
Name: concept_class_id_1, dtype: int64

Ingredient            84883
Precise Ingredient       25
Name: concept_class_id_2, dtype: int64

Has form


...

Precise Ingredient    2891
Name: concept_class_id_2, dtype: int64

Has precise ing


,concept_class_id_1,concept_class_id_2,concept_code_1,concept_code_2,concept_id_1,concept_id_2,concept_name_1,concept_name_2,relationship_id_12
2144,Clinical Drug Comp,Precise Ingredient,316779,91603,561425,561401,"Tetanus immune globulin, human 250 UNT/ML","Tetanus immune globulin, human",Has precise ing
2147,Brand Name,Precise Ingredient,1087392,91603,40234560,561401,Hypertet,"Tetanus immune globulin, human",Has precise ing
17576,Clinical Drug Comp,Precise Ingredient,2167146,2167145,789802,789801,Immune Globulin Subcutaneous (Human) - hipp 16...,Immune Globulin Subcutaneous (Human) - hipp,Has precise ing
17577,Brand Name,Precise Ingredient,2167148,2167145,789803,789801,Cutaquig,Immune Globulin Subcutaneous (Human) - hipp,Has precise ing
23846,Clinical Drug Comp,Precise Ingredient,1946981,1946964,792417,792679,influenza A virus A/Slovenia/2903/2015 (H1N1) ...,influenza A virus A/Slovenia/2903/2015 (H1N1) ...,Has precise ing


Brand Name            8379
Clinical Drug Comp    4605
Name: concept_class_id_1, dtype: int64

Precise Ingredient    12937
Ingredient               47
Name: concept_class_id_2, dtype: int64

what are all the tables that convert to brand name?

In [91]:
term='Brand Name'
grep='[Hh]as'
rids=cr_verbose[cr_verbose.relationship_id_12.str.contains(grep)].relationship_id_12.unique()
for rid in rids:
    tmp = cr_verbose.query('relationship_id_12==@rid')
    if np.any(tmp.concept_class_id_2.str.contains(term)):
        print(rid)
        display(tmp.head())
        display(tmp.concept_class_id_1.value_counts())
        display(tmp.concept_class_id_2.value_counts())

Has brand name


,concept_class_id_1,concept_class_id_2,concept_code_1,concept_code_2,concept_id_1,concept_id_2,concept_name_1,concept_name_2,relationship_id_12
17443,Branded Drug Comp,Brand Name,2166575,2166574,789775,789774,thyrotropin alfa 1.1 MG [Thyrogen],Thyrogen,Has brand name
17444,Branded Drug Form,Brand Name,2166576,2166574,789776,789774,thyrotropin alfa Injection [Thyrogen],Thyrogen,Has brand name
17445,Branded Dose Group,Brand Name,2166577,2166574,789777,789774,Thyrogen Injectable Product,Thyrogen,Has brand name
17446,Branded Drug,Brand Name,2166578,2166574,789778,789774,thyrotropin alfa 1.1 MG Injection [Thyrogen],Thyrogen,Has brand name
17447,Branded Pack,Brand Name,2166579,2166574,790201,789774,{2 (thyrotropin alfa 1.1 MG Injection [Thyroge...,Thyrogen,Has brand name


Branded Drug          23439
Ingredient            22793
Branded Drug Comp     22511
Branded Dose Group    21600
Branded Drug Form     17854
Quant Branded Drug     2628
Branded Pack            431
Name: concept_class_id_1, dtype: int64

Brand Name    111256
Name: concept_class_id_2, dtype: int64

which tables convert the clinical drugs?

In [90]:
term='Clinical'
grep='[Hh]as'
rids=cr_verbose[cr_verbose.relationship_id_12.str.contains(grep)].relationship_id_12.unique()
for rid in rids:
    tmp = cr_verbose.query('relationship_id_12==@rid')
    if np.any(tmp.concept_class_id_1.str.contains(term)):
        print(rid)
        display(tmp.head())
        display(tmp.concept_class_id_1.value_counts())
        display(tmp.concept_class_id_2.value_counts())

RxNorm has ing


,concept_class_id_1,concept_class_id_2,concept_code_1,concept_code_2,concept_id_1,concept_id_2,concept_name_1,concept_name_2,relationship_id_12
2,Clinical Drug Comp,Ingredient,353465,26744,501345,501343,hepatitis B immune globulin 217 UNT/ML,hepatitis B immune globulin,RxNorm has ing
3,Clinical Drug Comp,Ingredient,825152,26744,501421,501343,hepatitis B immune globulin 220 UNT/ML,hepatitis B immune globulin,RxNorm has ing
8,Clinical Drug Comp,Ingredient,440313,26744,19110172,501343,hepatitis B immune globulin 200 UNT/ML,hepatitis B immune globulin,RxNorm has ing
9,Clinical Drug Comp,Ingredient,440315,26744,19110173,501343,hepatitis B immune globulin 50 UNT/ML,hepatitis B immune globulin,RxNorm has ing
10,Clinical Drug Comp,Ingredient,705903,26744,19126484,501343,hepatitis B immune globulin 312 UNT/ML,hepatitis B immune globulin,RxNorm has ing


Clinical Drug Comp     29575
Clinical Dose Group    29401
Clinical Drug Form     25932
Name: concept_class_id_1, dtype: int64

Ingredient            84883
Precise Ingredient       25
Name: concept_class_id_2, dtype: int64

Has tradename


...

Dose Form Group    38320
Name: concept_class_id_2, dtype: int64

Has component


,concept_class_id_1,concept_class_id_2,concept_code_1,concept_code_2,concept_id_1,concept_id_2,concept_name_1,concept_name_2,relationship_id_12
1364577,Clinical Drug,Clinical Drug Form,199442,373831,19021828,40079589,butylscopolamine bromide 10 MG Oral Tablet,Scopolamine Oral Tablet,Has component


Clinical Drug    1
Name: concept_class_id_1, dtype: int64

Clinical Drug Form    1
Name: concept_class_id_2, dtype: int64

which tables converts the Quants?

In [89]:
term='Quant'
grep='[Hh]as'
rids=cr_verbose[cr_verbose.relationship_id_12.str.contains(grep)].relationship_id_12.unique()
for rid in rids:
    tmp = cr_verbose.query('relationship_id_12==@rid')
    if np.any(tmp.concept_class_id_1.str.contains(term)):
        print(rid)
        display(tmp.head())
        display(tmp.concept_class_id_1.value_counts())
        display(tmp.concept_class_id_2.value_counts())

Has tradename


,concept_class_id_1,concept_class_id_2,concept_code_1,concept_code_2,concept_id_1,concept_id_2,concept_name_1,concept_name_2,relationship_id_12
40,Quant Clinical Drug,Quant Branded Drug,825024,825138,19133078,501415,0.5 ML hepatitis B immune globulin 217 UNT/ML ...,0.5 ML hepatitis B immune globulin 217 UNT/ML ...,Has tradename
70,Clinical Drug,Branded Drug,825153,825156,501482,501483,hepatitis B immune globulin 220 UNT/ML Injecta...,hepatitis B immune globulin 220 UNT/ML Injecta...,Has tradename
82,Quant Clinical Drug,Quant Branded Drug,825157,825159,501484,501485,0.5 ML hepatitis B immune globulin 220 UNT/ML ...,0.5 ML hepatitis B immune globulin 220 UNT/ML ...,Has tradename
93,Quant Clinical Drug,Quant Branded Drug,825160,825161,19133085,501487,1 ML hepatitis B immune globulin 220 UNT/ML Pr...,1 ML hepatitis B immune globulin 220 UNT/ML Pr...,Has tradename
205,Clinical Drug,Branded Drug,251817,545642,509103,509106,meningococcal group A polysaccharide 0.1 MG/ML...,meningococcal group A polysaccharide 0.1 MG/ML...,Has tradename


Clinical Drug Comp     34597
Clinical Drug          23601
Clinical Dose Group    21624
Clinical Drug Form     17866
Quant Clinical Drug     2681
Clinical Pack            802
Name: concept_class_id_1, dtype: int64

Branded Drug Comp     34597
Branded Drug          23601
Branded Dose Group    21624
Branded Drug Form     17866
Quant Branded Drug     2681
Branded Pack            802
Name: concept_class_id_2, dtype: int64

Has brand name


...

Brand Name    111256
Name: concept_class_id_2, dtype: int64

RxNorm has dose form


,concept_class_id_1,concept_class_id_2,concept_code_1,concept_code_2,concept_id_1,concept_id_2,concept_name_1,concept_name_2,relationship_id_12
78843,Clinical Drug Form,Dose Form,1998450,1998426,963795,963781,aripiprazole Oral Tablet with Sensor,Oral Tablet with Sensor,RxNorm has dose form
78844,Branded Drug Form,Dose Form,1998452,1998426,963797,963781,aripiprazole Oral Tablet with Sensor [Abilify],Oral Tablet with Sensor,RxNorm has dose form
181300,Clinical Drug,Dose Form,252559,2107948,939359,1356078,Budesonide 0.5 MG/ML Inhalation Suspension,Inhalation Suspension,RxNorm has dose form
181301,Branded Drug,Dose Form,616817,2107948,939542,1356078,Budesonide 0.25 MG/ML Inhalation Suspension [P...,Inhalation Suspension,RxNorm has dose form
181302,Branded Drug,Dose Form,616819,2107948,939543,1356078,Budesonide 0.5 MG/ML Inhalation Suspension [Pu...,Inhalation Suspension,RxNorm has dose form


Clinical Drug          37378
Branded Drug           23620
Branded Drug Form      17854
Clinical Drug Form     16074
Quant Clinical Drug     3294
Quant Branded Drug      2672
Branded Pack             797
Clinical Pack            703
Name: concept_class_id_1, dtype: int64

Dose Form    102392
Name: concept_class_id_2, dtype: int64